In [ ]:
#Update your token
STUDENT_TOKEN = 'JOSE MIGUEL'

In [ ]:
## ignore this code, just used for submission
import requests
import pprint
import json
import random
import time
from copy import copy, deepcopy

class Game:
  def __init__(self, state, status, player):
    self.state = state
    self.status = status
    self.player = player

  def is_waiting(self):
    return self.status == 'waiting'

  def is_end(self):
    return self.status == 'complete'

  def get_board(self):
    print(self.state)
    return json.loads(self.state)

  def actions(self):
    return []

  def print_game(self):
    print(self.state)

def new_game(game_type, multi_player = False):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/new-game?TOKEN=%s&game-type=%s&multi-player=%s' % (STUDENT_TOKEN, game_type, 'True' if multi_player else 'False'))
    if r.status_code == 200:
      return r.json()['game-id']
    print(r.content)

def join_game(game_type, game_id):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/join-game?TOKEN=%s&game-type=%s&game-id=%s' % (STUDENT_TOKEN, game_type, game_id))
    if r.status_code == 200:
      return r.json()['player']
    print(r.content)

def game_state(game_type, game_id, GameClass):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/game-state?TOKEN=%s&game-type=%s&game-id=%s' % (STUDENT_TOKEN, game_type, game_id))
    if r.status_code == 200:
      return GameClass(r.json()['state'], r.json()['status'], r.json()['player'])
    print(r.content)

def update_game(game_type, game_id, player, move):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/update-game?TOKEN=%s&game-type=%s&game-id=%s&player=%s&move=%s' % (STUDENT_TOKEN, game_type, game_id, player, move))
    if r.status_code == 200:
      return r.content
    print(r.content)

def game_loop(solver, GameClass, game_type, multi_player=False, id=None):
    while id == None:
        print('\033[92mCreating new game...\033[0m')
        id = new_game(game_type, multi_player)

    print('\033[92mJoining game with id: %s\033[0m' % id)
    player = join_game(game_type, id)

    print('\033[92mPlaying as %s\033[0m' % player)

    game = game_state(game_type, id, GameClass)
    print('\033[91mWaiting for the other player to join...\033[0m')
    while game.is_waiting():
        time.sleep(10)
        game = game_state(game_type, id, GameClass)

    while True:
        game = game_state(game_type, id, GameClass)
        game.print_game()
        if game.is_end():
            if game.player == '-':
                print('\033[94mdraw\033[0m')
                return 'Draw'
            else:
                if game.player == player:
                    print('\033[92mYou won\033[0m')
                    return 'You won'
                else:
                    print('\033[91mYou lost\033[0m')
                    return 'You lost'
        if game.player == player:
            print('Making next move...')
            move = solver(game)
            update_result = update_game(game_type, id, player, json.dumps(move))
            print(update_result)
        else:
            time.sleep(2)

In [ ]:
from functools import reduce
from copy import copy, deepcopy
import json

class Stratego(Game):
    def __init__(self, state, status, player):
        Game.__init__(self, state, status, player)

    def actions(self):
        return self.state['possible_actions']

    def card_str(self, card):
        return '+'.join(map(str, card))

    def print_game(self):
        print('Board: ' + "\n".join("".join(row) for row in self.state['board']))
        pprint.pprint(self.state)
        if self.state['past_actions']:
            print('Last action: ' + str(self.state['past_actions'][-1]))
            print('Last two actions: ' + " ".join(str(action) for action in self.state["past_actions"][-2:]))

    def other_player(self):
        if self.player == 'X': return 'O'
        if self.player == 'O': return 'X'
    def get_board(self):
        return self.state


from enum import Enum
from copy import deepcopy

class Game_Pieces(Enum):
    Bomb = 'B'
    Flag = 'f'
    Empty = '_'
    Lake = 'L'
    Spy = 's'
    Scout = 'S'
    General = 'G'
    Field_Marshal = 'F'
    Miner = 'M'
    Unknown = 'X'

from typing import List, Optional
import random

class StrategoBot:
    def __init__(self):
        # Assign heuristic values for each piece
        self.PIECE_VALUES = {
            Game_Pieces.Flag: 0,
            Game_Pieces.Bomb: -1,
            Game_Pieces.Empty: 0,
            Game_Pieces.Lake: -2,
            Game_Pieces.General: 9,
            Game_Pieces.Spy: 1,
            Game_Pieces.Scout: 2,
            Game_Pieces.Field_Marshal: 10,
            Game_Pieces.Miner: 7,
            Game_Pieces.Unknown: -3
        }

        # Map string
        self.Pieces = {
            'B': Game_Pieces.Bomb,
            'f': Game_Pieces.Flag,
            '_': Game_Pieces.Empty,
            'L': Game_Pieces.Lake,
            'G': Game_Pieces.General,
            's': Game_Pieces.Spy,
            'S': Game_Pieces.Scout,
            'F': Game_Pieces.Field_Marshal,
            'M': Game_Pieces.Miner,
            'O': Game_Pieces.Unknown,
            'X': Game_Pieces.Unknown
        }

    def Check_Move(self, game_state: dict, move: List[List[int]]) -> float:
        board = game_state['board']
        start_position, new_position = move
        piece = board[start_position[0]][start_position[1]]
        target = board[new_position[0]][new_position[1]]

        score = 0.0

        # Score for attacking
        if target == 'O':
            if piece == 's':
                score += 8
            elif piece == 'M':
                score += 5
            else:
                score += 3

        if piece in ['F', 'G', 'M']:
            score += (new_position[0] - start_position[0]) * 0.5

        if piece == 'S':
            distance = abs(new_position[0] - start_position[0]) + abs(new_position[1] - start_position[1])
            score += distance * 0.3

        if piece == 'F':
            flag_position = self.check_piece(board, 'f')
            if flag_position:
                distance_to_flag = self.manhattan_heuristic(new_position, flag_position)
                score += (8 - distance_to_flag) * 0.4
        return score

    # manhattan distance
    def manhattan_heuristic(self, position1: List[int], position2: List[int]) -> int:
        return abs(position1[0] - position2[0]) + abs(position1[1] - position2[1])

    def check_piece(self, board: List[List[str]], piece: str) -> Optional[List[int]]:
        for i in range(len(board)):
            for j in range(len(board[i])):
                if board[i][j] == piece:
                    return [i, j]
        return None

    def Pick_move(self, game_state: dict) -> List[List[int]]:
        possible_actions = game_state['possible_actions']

        if not possible_actions:
            return None

        # Evaluation of moves
        move_scores = [(move, self.Check_Move(game_state, move))
                      for move in possible_actions]

        # Randomize from the top 3 best moves to avoid predictability
        best_moves = sorted(move_scores, key=lambda x: x[1], reverse=True)[:3]
        selected_move = random.choice(best_moves)[0]
        return selected_move

# Solution
def solution(game: Stratego) -> List[List[int]]:
    if game.state['phase'] == "place":
        if game.player == 'X':
            positions = [(y, x) for x in range(2, 6) for y in range(2)]
        else:
            positions = [(7 - y, x) for x in range(2, 6) for y in range(2)]
        pieces = ['F', 'G', 'M', 'S', 'S', 's', 'B', 'f']
        random.shuffle(pieces)
        final_pos = []
        for pos in positions:
            final_pos.append((pos[0], pos[1], pieces.pop()))
        print(final_pos)
        return final_pos

    bot = StrategoBot()
    game_state = game.get_board()
    return bot.Pick_move(game_state)

# minimax solution
def minimax_solution(game: Stratego) -> List[List[int]]:
    if game.state['phase'] == "place":

        if game.player == 'O':
            positions = [(y, x) for x in range(2, 6) for y in range(2)]
        else:
            positions = [(7 - y, x) for x in range(2, 6) for y in range(2)]
        pieces = ['F', 'G', 'M', 'S', 'S', 's', 'B', 'f']
        random.shuffle(pieces)
        final_pos = []
        for pos in positions:
            final_pos.append((pos[0], pos[1], pieces.pop()))
        print(final_pos)
        return final_pos

    # minimax bot for move selection
    bot = StrategoBot()
    game_state = game.get_board()
    return bot.Pick_move(game_state)


In [ ]:
game_loop(solution, Stratego, 'stratego', multi_player=False, id=None)

In [ ]:
# Success Rate calculator
wins = 0
losses = 0
draws = 0
num_games = 10

for i in range(num_games):
    print(f"\nStarting game {i+1}...")
    result = game_loop(solution, Stratego, 'stratego', multi_player=False, id=None)
    if result == 'You won':
        wins += 1
    elif result == 'You lost':
        losses += 1
    elif result == 'Draw':
        draws += 1
    else:
        print(f"Unexpected result: {result}")

print(f"\nResults after {num_games} games:")
print(f"Wins: {wins}")
print(f"Losses: {losses}")
print(f"Draws: {draws}")
win_rate = (wins / num_games) * 100
print(f"Win Rate: {win_rate}%")

In [ ]:
def random_solver(game):
    if game.state['phase'] == "place":
        if game.player == 'X':
            positions = [(y, x) for x in range(2, 6) for y in range(2)]
        else:
            positions = [(7 - y, x) for x in range(2, 6) for y in range(2)]
        pieces = ['F', 'G', 'M', 'S', 'S', 's', 'B', 'f']
        random.shuffle(pieces)
        final_pos = []
        for pos in positions:
            final_pos.append((pos[0], pos[1], pieces.pop()))
        print(final_pos)
        return final_pos
    else:
        return random.choice(game.actions())